In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install --upgrade gspread

import gspread
from oauth2client.service_account import ServiceAccountCredentials
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
creds = ServiceAccountCredentials.from_json_keyfile_name('/content/drive/My Drive/credentials.json', scope)
client = gspread.authorize(creds)

spreadsheet = client.open_by_url("https://docs.google.com/spreadsheets/d/1v625jAYEpZNoBtZSUhEcjsd-7SdqRlna_ZkbXf-UPCs/edit?gid=389550033")
sheet = spreadsheet.get_worksheet(0)

data = pd.DataFrame(sheet.get_all_records())

In [ ]:
data.head()
data.dtypes
data.isnull().sum() / len(data)

In [ ]:
subset_columns = [col for col in data.columns if col != "Мова програмування"]
print("Список колонок для видалення пропусків:", subset_columns)

data_cleaned = data.dropna(axis=1, how='any')

if "Мова програмування" in data_cleaned.columns:
    print("Колонка 'Мова програмування' залишилася")
else:
    print("Колонка 'Мова програмування' була видалена")

data_cleaned.isnull().sum() / len(data_cleaned)
data_cleaned.dropna(inplace=True)
data_cleaned.shape

In [ ]:
python_data = data_cleaned[data_cleaned['Мова програмування'].str.contains("Python", na=False)]
python_data.shape

In [ ]:
grouped_data = python_data.groupby("Посада")

print("Доступні колонки до групування:", data.columns)

grouped_data = data.groupby("Посада")

In [ ]:
salary_column = [col for col in data.columns if "Зарплата" in col]
print("Знайдена колонка для зарплати:", salary_column)

if salary_column:
    salary_stats = grouped_data[salary_column[0]].agg(['min', 'max']).reset_index()
    print(salary_stats)
else:
    print("Колонка з зарплатою не знайдена")

In [ ]:
def fill_avg_salary(row):
    return (row['min'] + row['max']) / 2

salary_stats['avg'] = salary_stats.apply(fill_avg_salary, axis=1)
salary_stats['avg'].describe()
salary_stats.to_csv('salary_stats.csv', index=False)

In [ ]:
position_counts = python_data['Посада'].value_counts()
plt.figure(figsize=(8, 8))
plt.pie(position_counts, labels=position_counts.index, autopct='%1.1f%%', startangle=140)
plt.title('Розподіл посад серед Python розробників')
plt.show()

In [ ]:
plt.figure(figsize=(10, 6))
sns.scatterplot(data=salary_stats, x='min', y='max', hue='Посада', palette='viridis', s=100)
plt.title('Скаттер-плот для мінімальної та максимальної зарплат')
plt.xlabel('Мінімальна зарплата')
plt.ylabel('Максимальна зарплата')
plt.show()

In [ ]:
plt.figure(figsize=(10, 6))
sns.barplot(x=position_counts.index, y=position_counts.values, palette='magma')
plt.title('Кількість розробників Python за посадою')
plt.xlabel('Посада')
plt.ylabel('Кількість розробників')
plt.xticks(rotation=45, ha='right')
plt.show()